## 소개

이 수업에서는 다음 내용을 다룹니다:  
- 함수 호출이란 무엇이며 그 사용 사례  
- OpenAI를 사용하여 함수 호출을 만드는 방법  
- 함수 호출을 애플리케이션에 통합하는 방법  

## 학습 목표

이 수업을 완료하면 다음을 알고 이해하게 됩니다:  

- 함수 호출을 사용하는 목적  
- OpenAI 서비스를 사용하여 함수 호출 설정하기  
- 애플리케이션 사용 사례에 맞는 효과적인 함수 호출 설계하기  


## 함수 호출 이해하기

이번 수업에서는 사용자가 챗봇을 통해 기술 과정을 찾을 수 있는 기능을 교육 스타트업에 구축하려고 합니다. 사용자의 기술 수준, 현재 역할 및 관심 기술에 맞는 과정을 추천할 것입니다.

이를 완성하기 위해 다음을 조합하여 사용할 것입니다:
 - `OpenAI`를 사용하여 사용자에게 채팅 경험을 제공
 - `Microsoft Learn Catalog API`를 사용하여 사용자의 요청에 따라 과정을 찾도록 지원
 - `Function Calling`을 사용하여 사용자의 쿼리를 받아 API 요청을 수행하는 함수로 전달

시작하기 위해, 먼저 왜 함수 호출을 사용하려는지 살펴보겠습니다:

print("다음 요청의 메시지:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # 함수 응답을 볼 수 있는 GPT로부터 새 응답 받기


print(second_response.choices[0].message)


### 함수 호출이 필요한 이유

이 과정의 다른 수업을 완료했다면, 아마도 대형 언어 모델(LLM)을 사용하는 강력함을 이해하고 있을 것입니다. 또한 그 한계점도 어느 정도 인지할 수 있을 것입니다.

함수 호출은 다음과 같은 문제를 해결하기 위해 OpenAI 서비스에 도입된 기능입니다:

응답 형식의 일관성 부족:
- 함수 호출 이전에는 대형 언어 모델의 응답이 구조화되어 있지 않고 일관성이 없었습니다. 개발자들은 출력의 각 변형을 처리하기 위해 복잡한 검증 코드를 작성해야 했습니다.

외부 데이터와의 제한된 통합:
- 이 기능이 도입되기 전에는 애플리케이션의 다른 부분에서 데이터를 채팅 컨텍스트에 통합하는 것이 어려웠습니다.

응답 형식을 표준화하고 외부 데이터와의 원활한 통합을 가능하게 함으로써, 함수 호출은 개발을 단순화하고 추가 검증 로직의 필요성을 줄여줍니다.

사용자들은 "스톡홀름의 현재 날씨는 어떻습니까?"와 같은 질문에 답변을 얻을 수 없었습니다. 이는 모델이 학습된 시점의 데이터에 한정되어 있었기 때문입니다.

아래 예제를 통해 이 문제를 살펴보겠습니다:

학생 데이터베이스를 만들어 적절한 강좌를 추천해주고자 한다고 가정해봅시다. 아래에는 포함된 데이터가 매우 유사한 두 학생에 대한 설명이 있습니다.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

우리는 이 데이터를 파싱하기 위해 LLM에 보낼 예정입니다. 이후 이 데이터를 API로 전송하거나 데이터베이스에 저장하는 데 사용할 수 있습니다.

LLM에게 우리가 관심 있는 정보를 지시하는 두 개의 동일한 프롬프트를 만들어 봅시다:


우리는 이 내용을 LLM에 보내서 우리 제품에 중요한 부분을 분석하고자 합니다. 그래서 LLM에 지시할 두 개의 동일한 프롬프트를 만들 수 있습니다:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


이 두 프롬프트를 만든 후, 우리는 `openai.ChatCompletion`을 사용하여 이를 LLM에 보낼 것입니다. 프롬프트는 `messages` 변수에 저장하고 역할은 `user`로 지정합니다. 이는 사용자가 챗봇에 메시지를 작성하는 것을 모방하기 위함입니다.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

이제 두 요청을 모두 LLM에 보내고 받은 응답을 검토할 수 있습니다.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

프롬프트는 동일하고 설명도 비슷하지만, `Grades` 속성의 형식은 다르게 나올 수 있습니다.

위 셀을 여러 번 실행하면 형식이 `3.7` 또는 `3.7 GPA`가 될 수 있습니다.

이는 LLM이 작성된 프롬프트 형태의 비구조화된 데이터를 받아서 역시 비구조화된 데이터를 반환하기 때문입니다. 데이터를 저장하거나 사용할 때 무엇을 기대할 수 있는지 알기 위해서는 구조화된 형식이 필요합니다.

함수 호출(functional calling)을 사용하면 구조화된 데이터를 받을 수 있습니다. 함수 호출을 사용할 때 LLM은 실제로 어떤 함수를 호출하거나 실행하지 않습니다. 대신, LLM이 응답할 때 따라야 할 구조를 만듭니다. 그런 다음 이러한 구조화된 응답을 사용하여 애플리케이션에서 어떤 함수를 실행할지 결정합니다.


![함수 호출 흐름도](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.ko.png)


그런 다음 함수에서 반환된 값을 가져와 이를 다시 LLM에 보낼 수 있습니다. 그러면 LLM은 자연어를 사용하여 사용자의 질문에 답변할 것입니다.


### 함수 호출 사용 사례

**외부 도구 호출**  
챗봇은 사용자 질문에 답변을 제공하는 데 뛰어납니다. 함수 호출을 사용하면 챗봇이 사용자 메시지를 활용해 특정 작업을 완료할 수 있습니다. 예를 들어, 학생이 챗봇에게 "이 과목에 대해 더 도움이 필요하다고 강사에게 이메일 보내줘"라고 요청할 수 있습니다. 이 경우 `send_email(to: string, body: string)` 함수를 호출할 수 있습니다.

**API 또는 데이터베이스 쿼리 생성**  
사용자는 자연어를 사용해 정보를 찾고, 이를 형식화된 쿼리나 API 요청으로 변환할 수 있습니다. 예를 들어, 교사가 "마지막 과제를 완료한 학생이 누구인가요?"라고 요청하면 `get_completed(student_name: string, assignment: int, current_status: string)`라는 함수를 호출할 수 있습니다.

**구조화된 데이터 생성**  
사용자는 텍스트 블록이나 CSV를 LLM을 사용해 중요한 정보를 추출할 수 있습니다. 예를 들어, 학생이 평화 협정에 관한 위키피디아 기사를 AI 플래시 카드로 변환할 수 있습니다. 이는 `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`라는 함수를 사용해 수행할 수 있습니다.


## 2. 첫 번째 함수 호출 만들기

함수 호출을 만드는 과정은 3가지 주요 단계로 구성됩니다:  
1. 함수 목록과 사용자 메시지를 포함하여 Chat Completions API를 호출하기  
2. 모델의 응답을 읽고 작업 수행, 즉 함수 또는 API 호출 실행하기  
3. 함수의 응답을 사용하여 사용자에게 응답을 생성하기 위해 Chat Completions API를 다시 호출하기.


![함수 호출의 흐름](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.ko.png)


### 함수 호출의 요소

#### 사용자 입력

첫 번째 단계는 사용자 메시지를 만드는 것입니다. 이는 텍스트 입력의 값을 동적으로 할당하거나 여기에서 값을 할당할 수 있습니다. Chat Completions API를 처음 사용하는 경우 메시지의 `role`과 `content`를 정의해야 합니다.

`role`은 `system`(규칙 생성), `assistant`(모델) 또는 `user`(최종 사용자) 중 하나일 수 있습니다. 함수 호출의 경우, 이를 `user`로 할당하고 예제 질문을 작성합니다.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### 함수 생성하기. 

다음으로 함수와 그 함수의 매개변수를 정의할 것입니다. 여기서는 `search_courses`라는 하나의 함수만 사용할 것이지만 여러 개의 함수를 만들 수도 있습니다.

**중요** : 함수는 LLM에 대한 시스템 메시지에 포함되며 사용 가능한 토큰 수에 포함됩니다.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**정의** 

함수 정의 구조는 여러 수준으로 구성되며, 각 수준마다 고유한 속성이 있습니다. 중첩 구조에 대한 개요는 다음과 같습니다:

**최상위 함수 속성:**

`name` - 호출하려는 함수의 이름입니다. 

`description` - 함수가 어떻게 작동하는지에 대한 설명입니다. 여기서는 구체적이고 명확하게 작성하는 것이 중요합니다. 

`parameters` - 모델이 응답에서 생성하기를 원하는 값과 형식의 목록입니다. 

**파라미터 객체 속성:**

`type` - 파라미터 객체의 데이터 유형(보통 "object")입니다.

`properties` - 모델이 응답에 사용할 특정 값들의 목록입니다. 

**개별 파라미터 속성:**

`name` - 속성 키에 의해 암묵적으로 정의됩니다(예: "role", "product", "level").

`type` - 이 특정 파라미터의 데이터 유형(예: "string", "number", "boolean")입니다. 

`description` - 특정 파라미터에 대한 설명입니다. 

**선택적 속성:**

`required` - 함수 호출을 완료하는 데 필요한 파라미터를 나열하는 배열입니다.


### 함수 호출하기  
함수를 정의한 후에는 이제 Chat Completion API 호출에 포함시켜야 합니다. 이를 위해 요청에 `functions`를 추가합니다. 이 경우 `functions=functions`입니다.  

또한 `function_call`을 `auto`로 설정하는 옵션도 있습니다. 이는 사용자가 직접 할당하는 대신 LLM이 사용자 메시지를 기반으로 어떤 함수를 호출할지 결정하도록 하는 것을 의미합니다.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

이제 응답을 살펴보고 어떻게 형식화되었는지 보겠습니다:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

함수 이름이 호출된 것을 볼 수 있으며, 사용자 메시지에서 LLM이 함수의 인수에 맞는 데이터를 찾을 수 있었습니다.


## 3.애플리케이션에 함수 호출 통합하기. 


LLM에서 포맷된 응답을 테스트한 후, 이제 이를 애플리케이션에 통합할 수 있습니다. 

### 흐름 관리하기 

이를 애플리케이션에 통합하기 위해 다음 단계를 진행해 봅시다: 

먼저, OpenAI 서비스에 호출을 하고 메시지를 `response_message`라는 변수에 저장합시다. 


In [ ]:
response_message = response.choices[0].message

이제 Microsoft Learn API를 호출하여 강의 목록을 가져오는 함수를 정의하겠습니다:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



모범 사례로, 모델이 함수를 호출하고자 하는지 확인합니다. 그 후, 사용 가능한 함수 중 하나를 생성하고 호출된 함수와 일치시킵니다.  
그 다음, 함수의 인수를 가져와 LLM의 인수에 매핑합니다.  

마지막으로, 함수 호출 메시지와 `search_courses` 메시지에서 반환된 값을 추가합니다. 이렇게 하면 LLM이 자연어를 사용하여 사용자에게 응답하는 데 필요한 모든 정보를 갖게 됩니다.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



이제 업데이트된 메시지를 LLM에 보내 자연어 응답을 받을 수 있도록 하겠습니다. API JSON 형식의 응답 대신에요.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## 코드 챌린지 

훌륭합니다! OpenAI 함수 호출 학습을 계속하려면 다음을 구축할 수 있습니다: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst 
 - 학습자가 더 많은 과정을 찾는 데 도움이 될 수 있는 함수의 추가 매개변수. 사용 가능한 API 매개변수는 여기에서 확인할 수 있습니다: 
 - 학습자의 모국어와 같은 더 많은 정보를 받는 또 다른 함수 호출 생성 
 - 함수 호출 및/또는 API 호출이 적합한 과정을 반환하지 않을 때의 오류 처리 생성


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 노력하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있음을 유의하시기 바랍니다. 원본 문서의 원어 버전이 권위 있는 출처로 간주되어야 합니다. 중요한 정보의 경우 전문적인 인간 번역을 권장합니다. 본 번역 사용으로 인한 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
